© 2024 Brian Butka. All rights reserved.

<a href="https://colab.research.google.com/github/bbutka/CEC220/blob/main/Bit_Pair_Recoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from termcolor import colored

def binary_to_decimal(binary_str):
    return int(binary_str, 2) if binary_str[0] == '0' else int(binary_str, 2) - (1 << len(binary_str))

def decimal_to_binary(n, bits):
    if n >= 0:
        s = bin(n & int("1"*bits, 2))[2:]
    else:
        s = bin((1 << bits) + n)[2:]
    return ("{0:0>%s}" % (bits)).format(s)

def bit_pair_encoding(binary):
    # Step 1: Add one bit to the left by sign extension if odd number of bits
    n = len(binary)
    if n % 2 != 0:
        binary = binary[0] + binary
        print(f"Sign extended binary: {binary}")

    # Step 2: Add a 0 to the right of the number
    binary = binary + '0'
    print(f"Binary with 0 added to the right: {binary}")

    # Step 3: Group in pairs starting from the left
    pairs = [binary[i:i+2] for i in range(0, len(binary), 2)]
    print(f"Grouped pairs: {pairs}")

    grouped_pairs_plus_right = []

    # Create the groups plus the bit to the right
    for i in range(len(pairs)-1):
        grouped_pairs_plus_right.append(pairs[i] + pairs[i+1][0])

    print(f"Grouped pairs plus bit to the right: {grouped_pairs_plus_right}")

    encoded_values = []

    # Bit pair encoding based on the provided table
    bit_pair_dict = {
        '000': 0,
        '001': 1,
        '010': 1,
        '011': 2,
        '100': -2,
        '101': -1,
        '110': -1,
        '111': 0
    }

    # Step 4: Apply bit pair encoding to each group
    for group in grouped_pairs_plus_right:
        encoded_value = bit_pair_dict[group]
        encoded_values.append(encoded_value)
        print(f"Group {group} encoded as {encoded_value}")

    return encoded_values

def binary_addition_with_carry(a, b):
    max_len = max(len(a), len(b))
    a = a.zfill(max_len)
    b = b.zfill(max_len)

    carry = 0
    result = ''
    carries = ''

    for i in range(max_len - 1, -1, -1):
        total = int(a[i]) + int(b[i]) + carry
        if total > 1:
            carry = 1
            carries = '1' + carries
        else:
            carry = 0
            carries = '0' + carries  # Change ' ' to '0' for explicit leading zeros
        result = str(total % 2) + result

    # Shift carries one position to the left and add a trailing zero
    carries = carries[1:] + '0'
    return result.zfill(max_len), carries.zfill(max_len)

def sign_extend(value, bits):
    if value[0] == '0':
        return value.zfill(bits)
    else:
        return value.rjust(bits, '1')

def binary_multiplication(M, Q):
    M_dec = binary_to_decimal(M)
    neg_M = decimal_to_binary(-M_dec, len(M))
    double_M = decimal_to_binary(2 * M_dec, len(M) + 1)
    neg_double_M = decimal_to_binary(-2 * M_dec, len(M) + 1)

    M_len = len(M)
    Q_len = len(Q)
    total_length = 2 * Q_len + M_len  # Ensure total length is 2 * number of digits in Q + number of bits in M

    # Sign extend M and neg_M to the total length needed
    M = sign_extend(M, total_length)
    neg_M = sign_extend(neg_M, total_length)
    double_M = sign_extend(double_M, total_length)
    neg_double_M = sign_extend(neg_double_M, total_length)

    partial_products = []

    # Calculate the partial products in the order Q[2]*M, Q[1]*M, Q[0]*M
    for i in range(Q_len):
        index = Q_len - 1 - i  # Process Q from right to left
        if Q[index] == 2:
            partial_product = double_M
        elif Q[index] == 1:
            partial_product = M
        elif Q[index] == -1:
            partial_product = neg_M
        elif Q[index] == -2:
            partial_product = neg_double_M
        else:
            partial_product = '0' * total_length
        # Shift and restrict the length to total_length
        partial_product = partial_product + '0' * (2 * i)
        partial_product = partial_product[-total_length:]
        partial_products.append(partial_product)

    # Ensure all partial products have the same number of bits
    partial_products = [sign_extend(pp, total_length) for pp in partial_products]

    # Initialize final product and carries
    final_product = '0' * total_length
    total_carries = '0' * total_length  # Initialize with '0' to ensure leading zeros are kept

    # Sum the partial products to get the final product
    for partial_product in partial_products:
        final_product, carries = binary_addition_with_carry(final_product, partial_product)
        total_carries, _ = binary_addition_with_carry(total_carries, carries)  # Replace space with '0'

    final_product_dec = binary_to_decimal(final_product)
    return M, Q, partial_products, final_product, final_product_dec, total_carries, total_length

def main():
    binary = input("Enter a binary number: ").strip()

    # Validate input
    if not all(bit in '01' for bit in binary):
        print("Invalid binary number. Please enter a valid binary number.")
        return

    encoded = bit_pair_encoding(binary)
    print(f"Bit-pair encoding of {binary} is: {encoded}")

    multiply = input("Do you wish to multiply this number by another binary number? (yes/no): ").strip().lower()

    if multiply == 'yes':
        Q = encoded  # Use the encoded values as the variable Q
        binary_M = input("Enter the binary number to multiply with: ").strip()
        M, Q, partial_products, final_product_bin, final_product_dec, total_carries, total_length = binary_multiplication(binary_M, Q)

        M_dec = binary_to_decimal(M)
        Q_dec = sum(q * (4 ** i) for i, q in enumerate(reversed(Q)))  # Calculate the decimal value for Q
        neg_M_binary = decimal_to_binary(-M_dec, len(M))  # Calculate binary representation of -M

        N = len(Q)  # Ensure N is properly defined

        t = 6  # Specify the number of columns to move to the right

        # Function to expand digits with spaces
        def expand_digits(binary_str):
            return " ".join([c for c in binary_str])

        # Calculate the required width for alignment
        max_len = total_length
        M_expanded = expand_digits(M).rjust(max_len)
        neg_M_expanded = expand_digits(neg_M_binary).rjust(max_len)
        Q_expanded = "".join([f"{q:+}" for q in Q]).rjust(max_len)

        # Display the multiplication problem
        print(f"\nMultiplication Problem:")
        print(f"{' ' * (t+0)}{M_expanded} (M = {M_dec}, -M = {neg_M_expanded})")
        print(f"{' ' * (t+7)}x {Q_expanded} (Q = {Q_dec})")
        print(f"{' ' * (t+6)}{'-' * max_len}")

        # Display the carries shifted one character to the left
        print(f"{' ' * t}{expand_digits(total_carries).rjust(max_len)} (carries)")

        # Display the partial products
        for i, partial_product in enumerate(partial_products):
            print(f"{' ' * t}{expand_digits(partial_product).rjust(max_len)} (partial product {i+1})")

        # Display the final result
        print(f"{' ' * t}{'-' * max_len}")
        print(f"{' ' * t}{expand_digits(final_product_bin).rjust(max_len)} (final product)")
        print(f"{' ' * t}Final product in decimal: {final_product_dec}")

if __name__ == "__main__":
    main()


Enter a binary number: 01101
Sign extended binary: 001101
Binary with 0 added to the right: 0011010
Grouped pairs: ['00', '11', '01', '0']
Grouped pairs plus bit to the right: ['001', '110', '010']
Group 001 encoded as 1
Group 110 encoded as -1
Group 010 encoded as 1
Bit-pair encoding of 01101 is: [1, -1, 1]
Do you wish to multiply this number by another binary number? (yes/no): yes
Enter the binary number to multiply with: 0111

Multiplication Problem:
      0 0 0 0 0 0 0 1 1 1 (M = 7, -M = 1 1 1 1 1 1 1 0 0 1)
             x     +1-1+1 (Q = 13)
            ----------
      1 1 1 1 0 0 1 0 0 0 (carries)
      0 0 0 0 0 0 0 1 1 1 (partial product 1)
      1 1 1 1 1 0 0 1 0 0 (partial product 2)
      0 0 0 1 1 1 0 0 0 0 (partial product 3)
      ----------
      0 0 0 1 0 1 1 0 1 1 (final product)
      Final product in decimal: 91
